<a href="https://colab.research.google.com/github/anand-sharan/big-data-challenge/blob/main/athletes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (124 kB/s

**Given the three sides of a triangle, calculate the area of the triangle using Heron's formula**

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-02-07 07:23:05--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.3’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.49MB/s    in 0.2s    

2021-02-07 07:23:05 (5.49 MB/s) - ‘postgresql-42.2.9.jar.3’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("L2").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Loading files into Spark DataFrame

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Camera_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
cameras_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Camera_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
cameras_df = cameras_df.dropna()
cameras_df = cameras_df.drop_duplicates()
cameras_df.show(10)


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   36712357|R1000XF1ASG0IH|B003ZWZVJA|     800620411|CowboyStudio 168 ...|          Camera|          3|            0|          1|   N|                Y|     Not as pictured|Great light for c...| 2012-01-04|
|         US|   43184205|R100HCQTI9KU03|B00HVLDRAQ|     547253826|Foscam FI9831W (B...|          Camera|          5|    

# Display the count of total records in the file

In [ ]:
cameras_df.count()

1801849

# Format data to load into database tables

In [ ]:
from pyspark.sql.functions import to_date
# Review DataFrame
reviews_id_df = cameras_df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-mm-dd').alias("review_date")])
reviews_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1000XF1ASG0IH|   36712357|B003ZWZVJA|     800620411| 2012-01-04|
|R100HCQTI9KU03|   43184205|B00HVLDRAQ|     547253826| 2014-01-12|
|R100X5DUIQ0WHZ|   46246471|B00DSN7A04|     168023778| 2015-01-15|
|R101ALJ70FC0SV|   50035679|B003Y2XN9G|     160318205| 2011-01-24|
|R101EXOPW5NQKG|   40870827|B0002Y5WZC|     755812374| 2009-01-31|
|R102GMDFYS7VZT|   35489344|B003VKYQQA|     294239880| 2013-01-28|
|R103N5C2JU237J|   12266905|B00006RH5I|     264608583| 2013-01-05|
|R1047HFDDD3CMC|   29495066|B009EOX2YY|     315702809| 2013-01-14|
|R104LU56P4RIR7|   35789310|B002K450QI|     729876602| 2014-01-22|
|R104RNS556K80C|    8231011|B003TQSHYI|     934248704| 2015-01-14|
|R105JR8LSCO2KO|   14925333|B0031AQ302|     876440535| 2010-01-04|
|R105R0JFYG76C1|   52742862|B003UOMWOK|     329117351| 2013-01

# Create Unique products dataframe

In [ ]:
products_df = cameras_df.select(["product_id", "product_title"]).drop_duplicates()
products_df.show(10)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B000TL3IPY|Calumet MF6829 Um...|
|B000JLXF4S|Seven Star SS212 ...|
|B00824PAQU|Westcott 577  Whi...|
|B00D699YZO|DRIFT HD GHOST & ...|
|B007OWOKQW|Fujifilm FinePix ...|
|B00004U0FD|Minox CD 140 35mm...|
|B002XJ3JL8|Visible Dust EZ S...|
|B00CGTN2N0|Queen Spy Mini U8...|
|B00004YKDQ|Canon PSC-100 Sof...|
|B0076UXKDG|Nikon MB-D12 Mult...|
+----------+--------------------+
only showing top 10 rows



# Create Reviews dataframe

In [ ]:
reviews_df = cameras_df.select(["review_id", "review_headline", "review_body"])
reviews_df.show(10)

+--------------+--------------------+--------------------+
|     review_id|     review_headline|         review_body|
+--------------+--------------------+--------------------+
|R1000XF1ASG0IH|     Not as pictured|Great light for c...|
|R100HCQTI9KU03|When you can't ge...|When you can't ge...|
|R100X5DUIQ0WHZ|Very sturdy & Hig...|Very high America...|
|R101ALJ70FC0SV|amazing   and wor...|I was amazed at t...|
|R101EXOPW5NQKG|Canon 200 DG Gadg...|Roomy, seems like...|
|R102GMDFYS7VZT|          Never came|It never came and...|
|R103N5C2JU237J|A must have gear ...|Very beautiful ca...|
|R1047HFDDD3CMC|Decent watch for ...|The watch, like t...|
|R104LU56P4RIR7|works great but t...|I realize this is...|
|R104RNS556K80C|            Good buy|Good clear protec...|
+--------------+--------------------+--------------------+
only showing top 10 rows



# Create Customers count dataframe

In [ ]:
customers_df = cameras_df.groupby("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show(10)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   40415719|             2|
|   52804016|             2|
|   12027367|             2|
|   17167957|             1|
|   37418901|             5|
|   49653393|             5|
|   19867350|             1|
|   49513864|             1|
|   50332375|             1|
|   30839996|             5|
+-----------+--------------+
only showing top 10 rows



# Create Vine dataframe

In [ ]:
vine_df = cameras_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show(10)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1000XF1ASG0IH|          3|            0|          1|   N|
|R100HCQTI9KU03|          5|            0|          1|   N|
|R100X5DUIQ0WHZ|          5|            0|          1|   N|
|R101ALJ70FC0SV|          4|            0|          0|   N|
|R101EXOPW5NQKG|          4|            0|          0|   N|
|R102GMDFYS7VZT|          1|            4|          6|   N|
|R103N5C2JU237J|          4|            1|          1|   N|
|R1047HFDDD3CMC|          3|            3|          5|   N|
|R104LU56P4RIR7|          4|            0|          0|   N|
|R104RNS556K80C|          5|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



# Push to AWS RDS instance

In [ ]:
mode = "append"
jdbc_url="jdbc:postgresql://mypostgresdb.cbogyvfe8ooe.us-west-1.rds.amazonaws.com:5432/my_data_class_db"
config = {"user":"<username>", "password": "<password>", "driver":"org.postgresql.Driver"}

In [ ]:
# Load reviews_df to table in RDS
reviews_id_df.write.jdbc(url=jdbc_url, table='junction_review_customer_product', mode=mode, properties=config)

In [ ]:
# Load products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Load customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# Load vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vines', mode=mode, properties=config)